In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/fq/0pnst_4s44b07dt0xbhvxlv80000gn/T/ipykernel_3834/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Importing modules

In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import os

from IPython.display import display
from IPython.display import clear_output

import sqlite3 as lite
from collections import OrderedDict
import time
import string
import re

import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.stem.porter import *

import logging

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

/var/folders/fq/0pnst_4s44b07dt0xbhvxlv80000gn/T/ipykernel_3834/321058008.py:33: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Creating Database connections

In [3]:
'''the path to the database'''
db_path1='TelegramData.db'

logging.info('Saving recipes to {}'.format(db_path1))
db = create_engine('sqlite:///{}'.format(db_path1)) 


### List of Relevant Channels from Classified_ChannelDetail table

In [4]:
validated_Channels=pd.read_sql("Classified_ChannelDetail",con=db)
validated_Channels=validated_Channels.loc[validated_Channels["Relevant?"]=="Yes"]
List_of_Valid_Channel=validated_Channels.Channel_ID.tolist()
#print(List_of_Valid_Channel)
list_6irish_channels = [-1001250192398,-1001486710610,-1001719423078, -1001522159637,-1001242115487,-1001198445854]
print(list_6irish_channels)

[-1001250192398, -1001486710610, -1001719423078, -1001522159637, -1001242115487, -1001198445854]


### Original Text Corpus 

In [5]:
text_content_1 = pd.read_sql("TextContent_Telegram_Channel",con=db)
# reading from the first table
text_content_1 = text_content_1.loc[text_content_1["Channel_ID"].isin(list_6irish_channels)]
#print(len(text_content_1))

print(text_content_1.head(5))

In [17]:
# reading from the second table
text_content_2 = pd.read_sql("TextContent_Telegram_Channel_2",con=db)
text_content_2 = text_content_2.loc[text_content_2["Channel_ID"].isin(list_6irish_channels)]
print(len(text_content_1))

205707


In [18]:
# merging the both table'''
TextCorpus = text_content_1.append(text_content_2)
print(len(TextCorpus))

208860


/var/folders/fq/0pnst_4s44b07dt0xbhvxlv80000gn/T/ipykernel_20762/1843917651.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TextCorpus = text_content_1.append(text_content_2)


### Cleaning Duplicate Entries

In [19]:
TextCorpus_unique = TextCorpus.drop_duplicates(['Channel_ID', 'MessageContent', 'MessageID', 'MessageDate', 'Message_FROm_UserID'], keep='first')

In [20]:
Text_Corpus_Refine = TextCorpus_unique
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.match(r'https?[:;]?/?/?\S*')) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("Tnx for joining")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("Thanks for joining")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("Thank you to all the new members for joining")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("Thank u x")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("Thanks for joining our small group")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("Thanks for Joining")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains(" has been banned! Reason: CAS ban")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("SEND OUR QR CODE")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("SHARE OUR QR CODE")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.contains("send our QR code")) ]
Text_Corpus_Refine = Text_Corpus_Refine.loc[ ~(Text_Corpus_Refine['MessageContent'].str.match("Check this out.."))]

In [21]:
Text_Corpus_Refine.to_sql("TelegramChannel_CovidOpinion_AcceptedTexts",con=db,index=False, if_exists="replace")

186083

In [22]:
len(Text_Corpus_Refine)

186083

## Stemming and Lemmatization

In [23]:
wnl = WordNetLemmatizer()
p_stemmer = PorterStemmer()


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    
    tag_dict = {"J": wn.ADJ,
                "N": wn.NOUN,
                "V": wn.VERB,
                "R": wn.ADV,
                "S": wn.ADJ_SAT}
    
    if tag in tag_dict.keys():
        return tag_dict.get(tag, tag_dict[tag])
  
    else:
        return tag_dict.get(tag, wn.NOUN)


        
def StemLemmatizerText(text_Content):
    h=list()
    h[:]=[]
   
    text_Content= text_Content.lower()
    
    text_Content = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text_Content)
    
    text_Content = re.sub('https?[:;]?/?/?\S*', ' ', text_Content)
    
    text_Content = re.sub(r'#|!|@|,|•|“|”|\"|\'|’s|n’t|’','',text_Content)
    
    text_Content = re.sub(r'\?',' ',text_Content)
    
    text_Content = re.sub('[.] *',' ',text_Content)
    
    text_Content = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|[0-9]?[0-9]?(\.[0-9][0-9]?)?|(\w+:\/\/\S+)|^rt", "", text_Content)
    
    text_Content = re.sub(r'com/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+[ ]?',' ',text_Content)


    
    for token in [wnl.lemmatize(w, pos=get_wordnet_pos(w)) for w in nltk.word_tokenize(text_Content)]:
        if wn.morphy(token):
            token=wn.morphy(token)
            
#         if get_wordnet_pos(token)==wn.NOUN:
#             token=p_stemmer.stem(token)

        if not token.isdigit() and token not in text.ENGLISH_STOP_WORDS:
            h.append(wnl.lemmatize(token,pos=get_wordnet_pos(token)))
       
    stemed_text_Content=''            
    stemed_text_Content=' '.join(h)
    return stemed_text_Content
    

print(StemLemmatizerText("AUSTRIA 🇦🇹: The Austrian government is in trouble? The obligation of imposing the control as directed by the minister of the interior is rejected by the police union https://infodujour.fr/politique/54068-lautriche-se-souleve-contre-la-dictature-sanitaire The union of Armed Forces have annnouced that they will participate in the grand protest in Vienna next Saturday https://twitter.com/Yoann_IV/status/1460687657999155205 The police and the army refuse to make controls for the sanitary pass under the name of ‘liberty and human dignity’. They will join a large protest against the mandatory lockdown of the non vaccinated on the 20th November in Vienna"))

print(StemLemmatizerText("Are you ready alpha men we will take everything and we will give them nothing 💪 https://t.me/alphamenassemblechannel"))

In [ ]:
Df2Process = pd.read_sql("TelegramChannel_CovidOpinion_AcceptedTexts",con=db)
print(len(Df2Process))
step=10000
begin=0
end=begin+step

while end<len(Df2Process):
    
    temp_df=Df2Process.loc[begin:end]
    temp_df["strip_text"]=''
    temp_df["semantic_unit_count"]=''

    for i in range(begin, end+1):
        if temp_df.at[i,"MessageContent"] and StemLemmatizerText(temp_df.at[i,"MessageContent"]):
            temp_df.at[i,"strip_text"] = StemLemmatizerText(temp_df.at[i,"MessageContent"])
            temp_df.at[i,"semantic_unit_count"]=len(nltk.word_tokenize(temp_df.at[i,"strip_text"]))
    
    temp_df.to_sql("Telegram_Channel_strippedText_Processed",con=db,index=False, if_exists="replace")
    
    begin=end+1
    end=end+step    
    del(temp_df)
    

    print(begin,end)

#quickFix for the leftover len(Df2Process)%tep
LO = pd.read_sql("Telegram_Channel_strippedText_Processed",con=db) 

temp_df = Df2Process.loc[len(LO)+1:len(Df2Process)-1]
temp_df["strip_text"]=''
temp_df["semantic_unit_count"]=''

for i in range(len(LO)+1, len(Df2Process)):
    if temp_df.at[i,"MessageContent"] and StemLemmatizerText(temp_df.at[i,"MessageContent"]):
        temp_df.at[i,"strip_text"]= StemLemmatizerText(temp_df.at[i,"MessageContent"])
        temp_df.at[i,"semantic_unit_count"]=len(nltk.word_tokenize(temp_df.at[i,"strip_text"]))
        

temp_df.to_sql("Telegram_Channel_strippedText_Processed",con = db, index=False, if_exists = "replace")

print("done")


## Remove Empty Enteries

In [26]:
temp_df = pd.read_sql("Telegram_Channel_strippedText_Processed",con=db)
print(len(temp_df))
temp_df = temp_df[temp_df['strip_text']!=""]
print(len(temp_df))

temp_df.to_sql("Telegram_Channel_strippedText_Processed", con=db, index=False, if_exists="replace")


176082
166653


166653